In [17]:
import pysam
sam_f = '/home/ytanigaw/data/nanopore/20161008_wgs_caucasian_48hr.10k.bwa.mapq60.20kb.chr11.sam'
ref_f = '/share/PI/mrivas/data/hg19/hg19.fa'
hg19 = pysam.FastaFile(ref_f)

In [18]:
def cigar2count(cigar_str, read, ref):
    '''
    parse CIGAR string and return summary stats
    
    need to expand to obtain a list of positions
    '''
    stats = {'M':0, 'I':0, 'D':0, 'N':0, 'S':0, 'H':0, 'P':0, '=':0, 'X':0}
    segment_start = 0
    i = 0
    relative_pos_on_read = 0
    relative_pos_on_ref = 0
    print len(read)
    while(i < len(cigar_str)):
        while('0' <= cigar_str[i] <= '9'):
            i = i + 1
        segment_length = int(cigar_str[segment_start:i])
        segment_type = cigar_str[i]
        relative_pos_on_read = relative_pos_on_read + segment_length
        if(segment_type in "MD"):
            relative_pos_on_ref = relative_pos_on_ref + segment_length
        if(segment_type == 'M'):
            #for aligned_pos in xrange(segment_length):
            #    if(read[relative_pos_on_read - aligned_pos] == 
            #       ref[relative_pos_on_ref - aligned_pos]):
            #        stats['='] = stats['='] + segment_length
            #    else:
            #        stats['X'] = stats['X'] + segment_length
            stats[segment_type] = stats[segment_type] + segment_length
        else:
            stats[segment_type] = stats[segment_type] + segment_length
        i = i + 1
        segment_start = i
    return(stats)

In [19]:
def read_sam(sam_f):
    '''
    read a sam file
    '''
    with open(sam_f, 'r') as f:
        head = f.readline().strip()
        entries = [line.strip().split() for line in f]
    return(head, entries)

In [20]:
def count(e, reference):
    ref = reference.fetch(reference = e[2], start = int(e[3]) - 1, end = int(e[3]) - 1 + len(e[9]))
    return(cigar2count(cigar_str=e[5], read = e[9], ref = ref))

In [21]:
def get_counts_main(sam_f, reference):
    head, entries = read_sam(sam_f)
    counts = [count(e, reference) for e in entries]
    return(head, entries, counts)

In [22]:
(head, entries, counts) = get_counts_main(sam_f=sam_f, reference = hg19)

30160
47925
33391
20086
26432
26674
20576


In [31]:
entries[0][9]

'...'

In [13]:
counts[0]

{'=': 0,
 'D': 1940,
 'H': 0,
 'I': 247,
 'M': 14563,
 'N': 0,
 'P': 0,
 'S': 15350,
 'X': 0}

In [24]:
counts[1]

{'=': 0,
 'D': 129,
 'H': 0,
 'I': 46,
 'M': 1134,
 'N': 0,
 'P': 0,
 'S': 46745,
 'X': 0}

In [32]:
counts

[{'=': 0,
  'D': 1940,
  'H': 0,
  'I': 247,
  'M': 14563,
  'N': 0,
  'P': 0,
  'S': 15350,
  'X': 0},
 {'=': 0,
  'D': 129,
  'H': 0,
  'I': 46,
  'M': 1134,
  'N': 0,
  'P': 0,
  'S': 46745,
  'X': 0},
 {'=': 0,
  'D': 173,
  'H': 0,
  'I': 40,
  'M': 1732,
  'N': 0,
  'P': 0,
  'S': 31619,
  'X': 0},
 {'=': 0,
  'D': 2078,
  'H': 0,
  'I': 641,
  'M': 19430,
  'N': 0,
  'P': 0,
  'S': 15,
  'X': 0},
 {'=': 0,
  'D': 485,
  'H': 0,
  'I': 157,
  'M': 4290,
  'N': 0,
  'P': 0,
  'S': 21985,
  'X': 0},
 {'=': 0,
  'D': 1856,
  'H': 0,
  'I': 1687,
  'M': 24919,
  'N': 0,
  'P': 0,
  'S': 68,
  'X': 0},
 {'=': 0,
  'D': 2113,
  'H': 0,
  'I': 560,
  'M': 19965,
  'N': 0,
  'P': 0,
  'S': 51,
  'X': 0}]

In [17]:
count(e)

{'=': 0,
 'D': 1940,
 'H': 0,
 'I': 247,
 'M': 14563,
 'N': 0,
 'P': 0,
 'S': 15350,
 'X': 0}

In [10]:
e = entries[0]

In [28]:
len(e[9])

30160

In [11]:
e[2]

'chr11'

In [23]:
int(e[3]) - 1

9162868

In [29]:
ref = hg19.fetch(reference = e[2], start = int(e[3]) - 1, end = int(e[3]) - 1 + len(e[9]))

## scratch code follows

In [5]:
entry[0]

'channel_75_80bdccec-0a01-48bb-854d-094281ccc083_qscore_8.2_read_score_-1.8'

In [6]:
seq = entry[9]

In [7]:
pos = int(entry[3]) - 1

In [8]:
pos

9162868

In [30]:
cigar = entry[5]

In [33]:
cigar2count(cigar)

{'=': 0,
 'D': 1940,
 'H': 0,
 'I': 247,
 'M': 14563,
 'N': 0,
 'P': 0,
 'S': 15350,
 'X': 0}

In [25]:
stats = {'M':0, 'I':0, 'D':0, 'N':0, 'S':0, 'H':0, 'P':0, '=':0, 'X':0}
start = 0
i = 0
relative_pos = 0
while(i < len(cigar_t)):
    while('0' <= cigar_t[i] <= '9'):
        i = i + 1
    segment_length = int(cigar_t[start:i])
    segment_type = cigar_t[i]
    relative_pos = relative_pos + segment_length
    stats[segment_type] = stats[segment_type] + segment_length
    #print(relative_pos, segment_length, segment_type)    
    i = i + 1
    start = i
      

(21, 21, 'M')
(22, 1, 'I')
(33, 11, 'M')
(34, 1, 'I')
(35, 1, 'M')
(36, 1, 'I')
(39, 3, 'M')
(42, 3, 'I')
(50, 8, 'M')
(51, 1, 'D')
(71, 20, 'M')
(72, 1, 'D')
(83, 11, 'M')
(85, 2, 'D')
(98, 13, 'M')
(99, 1, 'I')
(111, 12, 'M')
(113, 2, 'D')
(129, 16, 'M')
(133, 4, 'D')
(140, 7, 'M')
(141, 1, 'D')
(145, 4, 'M')
(148, 3, 'I')
(158, 10, 'M')
(159, 1, 'I')
(160, 1, 'M')
(161, 1, 'I')
(180, 19, 'M')
(181, 1, 'I')
(193, 12, 'M')
(194, 1, 'D')
(228, 34, 'M')
(230, 2, 'D')
(242, 12, 'M')
(243, 1, 'D')
(244, 1, 'M')
(246, 2, 'D')
(259, 13, 'M')
(261, 2, 'D')
(267, 6, 'M')
(268, 1, 'D')
(277, 9, 'M')


In [26]:
stats

{'=': 0, 'D': 20, 'H': 0, 'I': 13, 'M': 244, 'N': 0, 'P': 0, 'S': 0, 'X': 0}

In [43]:
for type in ['N', 'S', 'H', 'P', '=', 'X']:
    print(type, type in cigar)

('N', False)
('S', True)
('H', False)
('P', False)
('=', False)
('X', False)


In [40]:
cigar_t.split(['M', 'I', 'D'])

TypeError: expected a character buffer object

## reference sequence
- we can read reference sequence if needed

In [15]:
import pysam

In [17]:
ref_f = '/share/PI/mrivas/data/hg19/hg19.fa'

In [22]:
hg19 = pysam.FastaFile(ref_f)

In [24]:
ref = hg19.fetch(reference = 'chr11', start = pos, end = pos + len(seq))

In [26]:
len(ref)

30160